In [ ]:
import numpy as np
from io import StringIO
import io

In [ ]:
# Importando/Autenticando/Inicializando o API-GEE 
import ee #pip install earthengine-api
#from StringIO import StringIO
ee.Authenticate ()
ee.Initialize ()

In [ ]:
#Autorizando o acesso ao Google Drive
from google.colab import drive #conda install -c conda-forge google-colab
drive.mount('/content/drive')

In [ ]:
# Imprtando demais pacotes

import os
!pip install unidecode --quiet
import unidecode #!pip install unidecode --quiet
import glob
import pandas as pd

In [ ]:
def addHumidity(feature):
  # dew point (TD), temperature (T) 
  T = ee.Number(feature.get('temperature_2m')).subtract(273.15)

  TD = ee.Number(feature.get('dewpoint_temperature_2m')).subtract(273.15)

  c = ee.Number(243.04)

  b = ee.Number(17.625)

  # FORMULA: 100*Math.exp(c*b*(TD-T)/((c+T)*(c+TD)))

  return feature.set({'humidity': ee.Number(100).multiply((c.multiply(b).multiply(TD.subtract(T)).divide((c.add(T)).multiply(c.add(TD)))).exp())})  

def renameBand(image, band, new_band):
  return image.select ([band]).rename([new_band])

In [ ]:
def getMetric(y, collection, startDate, func, vars):
  start = ee.Date(startDate).advance(y, 'days')
  end = start.advance (23,'hours')
  
  if func == 'mean':
	  return collection.select(vars).filter(ee.Filter.date(start, end)).mean().set('Date', ee.Date(startDate).advance(y,'days')).set('system:time_start', ee.Date(startDate).advance(y,'days'))
		                 
  elif func == 'min':
    return collection.select(vars).filter(ee.Filter.date(start, end)).min().set('Date', ee.Date(startDate).advance(y,'days')).set('system:time_start', ee.Date(startDate).advance(y,'days'))
    
  elif func == 'max':
    return collection.select(vars).filter(ee.Filter.date(start, end)).max().set('Date', ee.Date(startDate).advance(y,'days')).set('system:time_start', ee.Date(startDate).advance(y,'days'))
   		
  elif func == 'sum':
    return collection.select(vars).filter(ee.Filter.date(start, end)).sum().set('Date', ee.Date(startDate).advance(y,'days')).set('system:time_start', ee.Date(startDate).advance(y,'days'))


In [ ]:
def concat_join (feature):
  return ee.Image.cat(feature.get('primary'), feature.get('secondary'))

In [ ]:
def get_era5land(startDate, endDate):

    # Difference between start and end in days 

    numberOfInstances = endDate.difference(ee.Date(startDate), 'day')

    seqInstances = ee.List.sequence(0, numberOfInstances.subtract(1))

 

    collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')

    vars = ee.List(['temperature_2m', 'dewpoint_temperature_2m', 'surface_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m'])

 

    era5_mean = ee.ImageCollection.fromImages(seqInstances.map(lambda y: getMetric(y, collection, startDate, 'mean', vars)))

 

    era5_min = ee.ImageCollection.fromImages(seqInstances.map(lambda y: getMetric(y, collection, startDate, 'min', 'temperature_2m'))).map(lambda image: renameBand(image, 'temperature_2m', 'min_temperature_2m'))

 

    era5_max = ee.ImageCollection.fromImages(seqInstances.map(lambda y: getMetric(y, collection, startDate, 'max', 'temperature_2m'))).map(lambda image: renameBand(image, 'temperature_2m', 'max_temperature_2m'))

 

    era5_sum = ee.ImageCollection.fromImages(seqInstances.map(lambda y: getMetric(y, collection, startDate, 'sum', 'total_precipitation')))

 

    # Define inner join

    innerJoin = ee.Join.inner(primaryKey='primary', secondaryKey='secondary')

    filterTimeEq = ee.Filter.equals(leftField='Date', rightField='Date')

 

    join = innerJoin.apply(era5_mean, era5_min, filterTimeEq).map(concat_join)

    mid_join = innerJoin.apply(join, era5_max, filterTimeEq).map(concat_join)

    era5_land = innerJoin.apply(mid_join, era5_sum, filterTimeEq).map(concat_join)

    return era5_land

In [ ]:
def featurize(feature, image):

    dict = {'RegionID': feature.get('RegionID'),

            'Date': ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')}

 

    properties = ['temperature_2m', 'min_temperature_2m', 'max_temperature_2m', 'dewpoint_temperature_2m',

                  'surface_pressure', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'total_precipitation']

    for property in properties:

        dict[property] = ee.List([feature.get(property), -999]).reduce(ee.Reducer.firstNonNull())

    return ee.Feature(None, dict)

 

def extract_var(image, geometries):

    stats = ee.Image(image).reduceRegions(**{'collection': geometries, "reducer": ee.Reducer.mean(), 'scale': 9000})

    return stats.map(lambda feature: featurize(feature, ee.Image(image)))

 

def getGeo(feature):

    return ee.Feature(feature.geometry(), {'RegionID': feature.get('CD_MUN')})

# Seconda Part

In [ ]:
startDate = ee.Date('2021-01-01')

endDate = ee.Date('2022-01-01')

 

# retrieve table with shapefiles info

table = ee.FeatureCollection("projects/earthengine-legacy/assets/users/sanchobuendia/BR")

 

# extract list of regions by code

regions = table.aggregate_array("CD_MUN").distinct()

In [ ]:
era5Land_daily = get_era5land(startDate, endDate)

In [ ]:
#print(era5Land_daily.getInfo())

In [ ]:
geometries = ee.FeatureCollection(table.map(getGeo)) #.filterMetadata('CD_MUN', 'starts_with', '1100130')

print(geometries.size().getInfo()) # VER

folderName = 'Brazil_2015_2021'

os.makedirs(folderName, exist_ok=True)

In [ ]:
# daily stats from ERA5-Land

era5LandStats = era5Land_daily.map(lambda image: extract_var(image, geometries)).flatten().map(addHumidity)

In [ ]:
# Export timeseries to Google Drive

task = ee.batch.Export.table.toDrive(**{

        'collection': era5LandStats,

        'description': 'ERA5land_daily_Brazil_2021',

        'fileFormat': 'CSV',

        'folder': folderName

        })

 

# run tasks automatically

task.start()

print(task.status())